In [ ]:
# --------------------- 1) LOAD + PARSE LOB (robust) ---------------------
import pandas as pd, numpy as np, re

if not LOB_PATH.exists():
    raise FileNotFoundError(f"LOB file not found: {LOB_PATH}")

print(f"Loading LOB: {LOB_PATH}")
lob = pd.read_csv(LOB_PATH)

# drop index column if present
if "Unnamed: 0" in lob.columns:
    lob = lob.drop(columns=["Unnamed: 0"])

# 1) Prefer a Unix-ms column (often named "0") -> 13-digit integers
ms_cols = [c for c in lob.columns
           if lob[c].astype(str).str.fullmatch(r"\d{13}").mean() > 0.8]

if ms_cols:
    # use the first ms column found
    ms_col = ms_cols[0]
    lob["ts"] = pd.to_datetime(lob[ms_col].astype("int64"), unit="ms", utc=True)
else:
    # 2) Else, look for a single column that already parses as full datetime
    dt_cols = []
    for c in lob.columns[:10]:
        parsed = pd.to_datetime(lob[c].astype(str), errors="coerce", utc=True)
        if parsed.notna().mean() > 0.8:
            dt_cols.append(c)
    if dt_cols:
        lob["ts"] = pd.to_datetime(lob[dt_cols[0]].astype(str), errors="coerce", utc=True)
    else:
        # 3) Else, detect separate date + time columns and combine
        def looks_like_date(s: pd.Series): 
            t = s.astype(str).head(20)
            return t.str.contains(r"^\d{4}-\d{2}-\d{2}$").mean() > 0.6
        def looks_like_time(s: pd.Series):
            t = s.astype(str).head(20)
            return t.str.contains(r"^\d{2}:\d{2}:\d{2}$").mean() > 0.6

        date_col = next((c for c in lob.columns[:10] if looks_like_date(lob[c])), None)
        time_col = next((c for c in lob.columns[:10] if looks_like_time(lob[c])), None)
        if not date_col or not time_col:
            raise KeyError("Could not detect timestamp: no unix-ms, full datetime, or separate date+time columns.")
        lob["ts"] = pd.to_datetime(
            lob[date_col].astype(str) + " " + lob[time_col].astype(str),
            utc=True, errors="coerce"
        )

# keep 2023 (your LOB window)
lob = lob.sort_values("ts").reset_index(drop=True)
lob = lob[(lob["ts"] >= "2023-01-01") & (lob["ts"] <= "2023-12-31 23:59:59")]

print("LOB window:", lob["ts"].min(), "->", lob["ts"].max(), f"rows={len(lob):,}")

# book columns start after the first 3 meta columns in your file
start_col_index = 3  # [0]=ms, [1]=date/full-dt, [2]=time/full-dt


Loading LOB: E:\DL Project\data\lob.csv
LOB window: 2023-01-09 22:17:40.926000+00:00 -> 2023-01-20 18:10:48.672000+00:00 rows=3,730,870


In [1]:
# === Bitcoin LOB + News Fusion (robust, Windows paths, UTC-safe) ===
# Install once:
#   pip install pandas numpy pyarrow fastparquet vaderSentiment
# Optional (better sentiment but slower):
#   pip install transformers torch

import pandas as pd
import numpy as np
import re
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# --------------------- CONFIG ---------------------
LOB_PATH  = Path(r"E:\DL Project\data\lob.csv")  # your LOB CSV (Jan 9–20, 2023)
NEWS_PATH = Path(r"E:\DL Project\data\news\bitcoin_sentiments_21_24.csv")
OUT_PATH  = Path(r"E:\DL Project\outputs\btc_lob_news_1min.parquet")

BUCKET    = "1min"   # resampling grid
LATENCY_S = 60       # news must be >=60s old by bucket end (anti-leakage)
LABEL_H   = 5        # 5-min ahead label

DATE_START = pd.Timestamp("2022-01-01", tz="UTC")
DATE_END   = pd.Timestamp("2024-12-31 23:59:59", tz="UTC")

USE_FINBERT   = False
FINBERT_MODEL = "ProsusAI/finbert"

# If your LOB has 3 meta columns then [ask_px, ask_sz, bid_px, bid_sz]*levels starts at index 3:
START_COL_INDEX = 3
TOP_K_LEVELS    = 10

# --------------------- HELPERS ---------------------
def to_dt_utc(x):
    """Parse timestamps robustly to UTC pandas.Timestamp."""
    if pd.isna(x): 
        return pd.NaT
    try:
        xi = int(x)
        unit = "ms" if xi > 1e12 else "s"
        return pd.to_datetime(xi, unit=unit, utc=True)
    except Exception:
        return pd.to_datetime(x, utc=True, errors="coerce")

def detect_date_time_cols(df, scan=12):
    """Return (date_col, time_col) if present."""
    date_col = None
    time_col = None
    for c in df.columns[:scan]:
        s = df[c].astype(str).head(50)
        if s.str.contains(r"^\d{4}-\d{2}-\d{2}$", regex=True).mean() > 0.7:
            date_col = c; break
    for c in df.columns[:scan]:
        s = df[c].astype(str).head(50)
        if s.str.contains(r"^\d{2}:\d{2}:\d{2}$", regex=True).mean() > 0.7:
            time_col = c; break
    return date_col, time_col

def build_tick_feats_from_pairs(df, k=10, start_col_index=3):
    """
    Your LOB rows repeat [ask_px, ask_sz, bid_px, bid_sz] per level after the first meta columns.
    """
    if df.shape[1] <= start_col_index:
        raise ValueError("Not enough columns in LOB to parse order book levels. Check start_col_index.")

    arr = df.iloc[:, start_col_index:].to_numpy(dtype=float)
    n_levels = min(arr.shape[1] // 4, k)
    if n_levels < 1:
        raise ValueError("Could not detect any price/size level blocks. Check your CSV structure.")

    apx = arr[:, 0::4][:, :n_levels]  # ask prices
    asz = arr[:, 1::4][:, :n_levels]  # ask sizes
    bpx = arr[:, 2::4][:, :n_levels]  # bid prices
    bsz = arr[:, 3::4][:, :n_levels]  # bid sizes

    best_bid = bpx[:, 0]
    best_ask = apx[:, 0]
    mid = (best_bid + best_ask) / 2.0
    spread = (best_ask - best_bid)
    spread_bps = 1e4 * spread / np.where(mid == 0, np.nan, mid)
    imb10 = (bsz.sum(axis=1) - asz.sum(axis=1)) / (bsz.sum(axis=1) + asz.sum(axis=1) + 1e-9)
    micro = (best_ask*bsz[:,0] + best_bid*asz[:,0]) / (bsz[:,0] + asz[:,0] + 1e-9)
    depth_sum = bsz.sum(axis=1) + asz.sum(axis=1)

    out = pd.DataFrame({
        "mid": mid,
        "spread_bps": spread_bps,
        "imb10": imb10,
        "micro": micro,
        "depth_sum": depth_sum,
    }, index=df.index)
    # simple OFI proxy
    out["ofi"] = pd.Series(out["micro"]).diff().fillna(0).values
    out["ts"]  = df["ts"].values
    return out

def to_utc_index(idx):
    """Ensure a DatetimeIndex is tz-aware UTC."""
    idx = pd.to_datetime(idx, errors="coerce")
    if getattr(idx, "tz", None) is None:
        return idx.tz_localize("UTC")
    else:
        return idx.tz_convert("UTC")

# --------------------- 1) LOAD + PARSE LOB ---------------------
if not LOB_PATH.exists():
    raise FileNotFoundError(f"LOB file not found: {LOB_PATH}")

print(f"Loading LOB: {LOB_PATH}")
lob = pd.read_csv(LOB_PATH)

# Drop obvious index columns if present
for col in ["Unnamed: 0", "index"]:
    if col in lob.columns:
        lob = lob.drop(columns=[col])

# Prefer a Unix-ms column if present (13-digit numeric-like)
ms_cols = [c for c in lob.columns if lob[c].astype(str).str.fullmatch(r"\d{13}").mean() > 0.8]
if ms_cols:
    ms_col = ms_cols[0]
    lob["ts"] = pd.to_datetime(lob[ms_col].astype("int64"), unit="ms", utc=True)
else:
    # Full datetime column?
    dt_cols = []
    for c in lob.columns[:12]:
        parsed = pd.to_datetime(lob[c].astype(str), errors="coerce", utc=True)
        if parsed.notna().mean() > 0.8:
            dt_cols.append(c)
    if dt_cols:
        lob["ts"] = pd.to_datetime(lob[dt_cols[0]].astype(str), errors="coerce", utc=True)
    else:
        # Separate date + time columns
        date_col, time_col = detect_date_time_cols(lob)
        if not date_col or not time_col:
            raise KeyError("Could not detect timestamp (no unix-ms, full datetime, or separate date+time).")
        lob["ts"] = pd.to_datetime(
            lob[date_col].astype(str) + " " + lob[time_col].astype(str),
            utc=True, errors="coerce"
        )

# Keep 2023 (your LOB window)
lob = lob.sort_values("ts").reset_index(drop=True)
lob = lob[(lob["ts"] >= "2023-01-01") & (lob["ts"] <= "2023-12-31 23:59:59")]
print("LOB window:", lob["ts"].min(), "->", lob["ts"].max(), f"rows={len(lob):,}")

# Build tick features from your column layout
lob_tick = build_tick_feats_from_pairs(lob, k=TOP_K_LEVELS, start_col_index=START_COL_INDEX)\
            .dropna(subset=["mid","micro"]).sort_values("ts")

# Resample to minute
g = lob_tick.set_index("ts").resample(BUCKET)
lob_1m = pd.DataFrame({
    "mid_last":        g["mid"].last(),
    "mid_mean":        g["mid"].mean(),
    "spread_bps_mean": g["spread_bps"].mean(),
    "imb10_mean":      g["imb10"].mean(),
    "micro_last":      g["micro"].last(),
    "ofi_sum":         g["ofi"].sum(),
    "depth_sum_mean":  g["depth_sum"].mean(),
})
lob_1m["micro_ret_1"] = lob_1m["micro_last"].pct_change()
lob_1m = lob_1m.dropna(subset=["mid_last"]).sort_index()
print("LOB_1m rows:", len(lob_1m), lob_1m.index.min(), "→", lob_1m.index.max())

# --------------------- 2) LOAD + PREP NEWS ---------------------
if not NEWS_PATH.exists():
    raise FileNotFoundError(f"News file not found: {NEWS_PATH}")

print(f"Loading News: {NEWS_PATH}")
news = pd.read_csv(NEWS_PATH)

# Detect datetime column in news
dt_cols = [c for c in news.columns if re.search(r"date|time|publish|created|datetime", c, re.I)]
if not dt_cols:
    raise KeyError("No datetime-like column found in NEWS CSV. Rename or point to the correct column.")
news["published_dt"] = news[dt_cols[0]].apply(to_dt_utc)

# Build text column robustly (title + body, or any available text-like columns)
title_col = next((c for c in news.columns if re.search(r"title|headline", c, re.I)), None)
body_col  = next((c for c in news.columns if re.search(r"body|content|article|summary|text", c, re.I)), None)

if title_col and body_col:
    news["text"] = (news[title_col].astype(str).fillna("") + " " +
                    news[body_col].astype(str).fillna("")).str.strip()
elif title_col:
    news["text"] = news[title_col].astype(str).fillna("").str.strip()
elif body_col:
    news["text"] = news[body_col].astype(str).fillna("").str.strip()
else:
    str_cols = [c for c in news.columns if news[c].dtype == "object"]
    if not str_cols:
        raise KeyError("No textual columns found in news CSV.")
    news["text"] = news[str_cols].astype(str).agg(" ".join, axis=1).str.strip()

# Filter date range + clean
news = news.dropna(subset=["published_dt"])
news = news[(news["published_dt"] >= DATE_START) & (news["published_dt"] <= DATE_END)]
news = news[news["text"].str.len() > 0].copy().sort_values("published_dt")
print("News rows in range:", len(news), news["published_dt"].min(), "→", news["published_dt"].max())

# --------------------- 3) SENTIMENT ---------------------
if USE_FINBERT:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import torch
    tok = AutoTokenizer.from_pretrained(FINBERT_MODEL)
    mdl = AutoModelForSequenceClassification.from_pretrained(FINBERT_MODEL); mdl.eval()
    def finbert_score(t):
        t = (t or "")[:1500]
        with torch.no_grad():
            enc = tok(t, return_tensors="pt", truncation=True)
            p = torch.softmax(mdl(**enc).logits, dim=-1).cpu().numpy()[0]  # [neg,neu,pos]
        return float(p[2] - p[0])  # pos - neg
    news["sent"] = news["text"].map(finbert_score)
else:
    vader = SentimentIntensityAnalyzer()
    news["sent"] = news["text"].map(lambda t: vader.polarity_scores(t)["compound"])

# Anti-leakage latency and bucketing
news["effective_dt"] = (news["published_dt"] + pd.to_timedelta(LATENCY_S, unit="s")).dt.floor(BUCKET)

news_agg = news.groupby("effective_dt").agg(
    sent_mean=("sent","mean"),
    sent_std =("sent","std"),
    sent_pos_share=("sent", lambda s: float(np.mean(s > 0))),
    news_count=("sent","count"),
).rename_axis("ts").sort_index()
news_agg["sent_std"] = news_agg["sent_std"].fillna(0)
print("News minute buckets:", len(news_agg), news_agg.index.min(), "→", news_agg.index.max())

# --------------------- 4) NORMALIZE INDICES TO UTC + JOIN + LABELS ---------------------
lob_1m.index   = to_utc_index(lob_1m.index)
news_agg.index = to_utc_index(news_agg.index)

X = lob_1m.join(
    news_agg,
    how="left"
).fillna({"sent_mean":0, "sent_std":0, "sent_pos_share":0, "news_count":0})

# Labels (5-min ahead)
H = LABEL_H
X["y_ret_5m"] = X["mid_last"].shift(-H)/X["mid_last"] - 1
X["y_dir_5m"] = np.sign(X["y_ret_5m"]).clip(-1, 1)

dataset = X.dropna(subset=["y_ret_5m"]).copy()

# --------------------- 5) SAVE ---------------------
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
dataset.to_parquet(OUT_PATH)
print(f"\nSaved {len(dataset):,} rows → {OUT_PATH}")
print(dataset.tail(3)[["mid_last","sent_mean","news_count","y_ret_5m","y_dir_5m"]])


Loading LOB: E:\DL Project\data\lob.csv
LOB window: 2023-01-09 22:17:40.926000+00:00 -> 2023-01-20 18:10:48.672000+00:00 rows=3,730,870


C:\Users\ONGC\AppData\Local\Temp\ipykernel_1520\818954630.py:160: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  lob_1m["micro_ret_1"] = lob_1m["micro_last"].pct_change()


LOB_1m rows: 15593 2023-01-09 22:17:00 → 2023-01-20 18:10:00
Loading News: E:\DL Project\data\news\bitcoin_sentiments_21_24.csv
News rows in range: 11158 2022-01-01 20:57:00+00:00 → 2024-09-12 00:00:00+00:00
News minute buckets: 9831 2022-01-01 20:58:00+00:00 → 2024-09-12 00:01:00+00:00

Saved 15,588 rows → E:\DL Project\outputs\btc_lob_news_1min.parquet
                           mid_last  sent_mean  news_count   y_ret_5m  \
ts                                                                      
2023-01-20 18:03:00+00:00    1.5570        0.0         0.0  14.530186   
2023-01-20 18:04:00+00:00    7.1195        0.0         0.0   1.969169   
2023-01-20 18:05:00+00:00    1.4460        0.0         0.0   0.266252   

                           y_dir_5m  
ts                                   
2023-01-20 18:03:00+00:00       1.0  
2023-01-20 18:04:00+00:00       1.0  
2023-01-20 18:05:00+00:00       1.0  


In [2]:
# ---------- DIAGNOSTICS -----------
import pandas as pd, numpy as np, re
from pathlib import Path

LOB_PATH = Path(r"E:/DL Project/data/lob.csv")
lob = pd.read_csv(LOB_PATH)
print("Columns (first 24):", lob.columns.tolist()[:24])
display(lob.head(6))

# show first 3 meta cols + first 12 book cols as raw numbers
print("\nSample raw numeric slice (first row):")
print(lob.iloc[0, :24].to_list())

# show some simple stats on first-level columns assuming start index 3
start = 3
cols = lob.columns.tolist()
first_level = cols[start:start+4]
print("\nAssumed first-level columns:", first_level)
sample = lob.iloc[:200, start:start+4].astype(float)
print("First-level sample stats (200 rows):")
print(sample.describe().T)


Columns (first 24): ['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,32,33,34,35,36,37,38,39,40,41
0,0,1673302660926,2023-01-09 22:17:40,17181.6,23.371,17181.5,0.746,17181.4,5.428,17181.2,...,17182.2,5.168,17182.3,0.02,17182.4,6.692,17182.5,1.904,17182.6,2.546
1,1,1673302661177,2023-01-09 22:17:41,17181.6,24.232,17181.5,0.694,17181.4,5.428,17181.2,...,17182.2,6.043,17182.3,0.02,17182.4,6.001,17182.5,1.869,17182.6,2.105
2,2,1673302661427,2023-01-09 22:17:41,17181.6,24.403,17181.5,0.694,17181.4,5.428,17181.2,...,17182.2,6.043,17182.3,0.02,17182.4,6.012,17182.5,1.869,17182.6,2.713
3,3,1673302661678,2023-01-09 22:17:41,17181.6,24.874,17181.5,0.694,17181.4,5.428,17181.2,...,17182.2,6.043,17182.3,0.02,17182.4,6.001,17182.5,2.570,17182.6,2.613
4,4,1673302661928,2023-01-09 22:17:41,17181.6,24.403,17181.5,0.694,17181.4,5.428,17181.2,...,17182.2,6.043,17182.3,0.02,17182.4,6.001,17182.5,2.589,17182.6,2.591
5,5,1673302662178,2023-01-09 22:17:42,17181.6,24.403,17181.5,0.686,17181.4,5.428,17181.2,...,17182.2,6.043,17182.3,0.02,17182.4,6.001,17182.5,3.785,17182.6,1.404



Sample raw numeric slice (first row):
[0, 1673302660926, '2023-01-09 22:17:40', 17181.6, 23.371, 17181.5, 0.746, 17181.4, 5.428, 17181.2, 0.89, 17181.1, 3.787, 17181.0, 0.908, 17180.9, 1.628, 17180.8, 0.007, 17180.7, 0.876, 17180.6, 2.854, 17181.7]

Assumed first-level columns: ['2', '3', '4', '5']
First-level sample stats (200 rows):
   count          mean        std        min        25%        50%  \
2  200.0  17183.546500   1.606891  17181.600  17181.800  17183.700   
3  200.0     27.380875  21.076859      1.657      4.895     24.335   
4  200.0  17183.446500   1.606891  17181.500  17181.700  17183.600   
5  200.0      3.011620   3.584985      0.004      1.035      1.070   

           75%        max  
2  17185.20000  17185.200  
3     50.74975     75.235  
4  17185.10000  17185.100  
5      3.82000     24.981  
